In [ ]:

from hamiltonian import Hamiltonian
from rgf import GreensFunction
import numpy as np
import matplotlib.pyplot as plt

In [ ]:

ham = Hamiltonian("one_d_wire")
gf = GreensFunction(ham)

energy = np.linspace(-5,5, 100)
dos = [None] * 100

for ide, e in enumerate(energy):
    dos[ide] = np.sum(gf.compute_density_of_states(E=e))
plt.plot(energy, dos)
plt.ylabel('DOS')
plt.xlabel('Energy')
plt.title('Energy vs DOS')

plt.show()

for ide, e in enumerate(energy):
    dos[ide] = gf.compute_transmission(E=e)
plt.plot(energy, dos)
plt.ylabel('Transmission')
plt.xlabel('Energy')
plt.title('Transmission vs energy')

plt.show()

In [ ]:
ham = Hamiltonian("qpc")
ham.Vg = 0.1
gf = GreensFunction(ham)
H00, H01, H10 = ham.get_H00_H01_H10()

print(H00.toarray())
print(H01.toarray())

for ide, e in enumerate(energy):
    dos[ide] = np.sum(gf.compute_density_of_states(E=e))
plt.plot(energy, dos)
plt.ylabel('DOS')
plt.xlabel('Energy')
plt.title('Energy vs DOS')

plt.show()

In [ ]:
import scipy.sparse as sp
from utils import smart_inverse

E = 100+1e-6j
H = ham.create_hamiltonian(blocks=False)
n = H.shape[0]

sigma_L = gf.lead_self_energy.self_energy("left", E, "sancho_rubio")
sigma_R = gf.lead_self_energy.self_energy("right", E, "sancho_rubio")


block_size = sigma_L.shape[0]

Sigma_L_full = sp.lil_matrix((n, n), dtype=complex)
Sigma_R_full = sp.lil_matrix((n, n), dtype=complex)
Sigma_L_full[:block_size, :block_size] = sigma_L
Sigma_R_full[-block_size:, -block_size:] = sigma_R
Sigma_L_full = Sigma_L_full.tocsc()
Sigma_R_full = Sigma_R_full.tocsc()

H_eff = H + Sigma_L_full + Sigma_R_full
A = E * sp.identity(n, dtype=complex, format='csc') - H_eff

# Use smart_inverse for efficient computation
G_R = smart_inverse(A)

if not sp.issparse(G_R):
    G_R = sp.csc_matrix(G_R)

Gamma_L = 1j * (Sigma_L_full - Sigma_L_full.conj().T)
Gamma_R = 1j * (Sigma_R_full - Sigma_R_full.conj().T)
print(Gamma_L.toarray())

T=(Gamma_L @ G_R @ Gamma_R @ G_R.conj().T)
print(T.toarray())



In [ ]:
A,B,C=gf.compute_central_greens_function(0, compute_lesser=False)
print(A)

In [ ]:
# Debug: Compare device boundary blocks and lead Hamiltonian
ham = Hamiltonian("qpc")
ham.W = 30
ham.L = 100
ham.t = 1.0
ham.o = 0.0
ham.Vg = 1.0

H = ham.create_hamiltonian(blocks=False).toarray()
print("Device first row block:")
print(H[:ham.W, :ham.W])
print("Device last row block:")
print(H[-ham.W:, -ham.W:])

H00, H01, H10 = ham.get_H00_H01_H10()
print("Lead H00:")
print(H00.toarray())
print("Lead H01:")
print(H01.toarray())

In [ ]:
# Sweep Fermi energy and plot transmission for QPC
ham = Hamiltonian("qpc")

ham.W = 10
ham.L = 30
ham.t = 1.0
ham.o = 0.0
ham.Vg = .1  # Moderate barrier

gf = GreensFunction(ham)

energies = np.linspace(-2, 2, 100)
transmission = []
for E in energies:
    transmission.append(gf.compute_transmission(E=E))

plt.plot(energies, transmission)
plt.xlabel('Energy')
plt.ylabel('Transmission')
plt.title('Transmission vs Energy for QPC')
plt.show()